# Treinamento e Validação da IA

Download das bibliotecas, configuração do ambiente e import das bibliotecas e ferramentas que serão usadas
Obs: Acesse Runtime e selecione a opção da GPU, para utilizar CUDA durante o treinamento da IA

In [ ]:
!pip uninstall -y pyarrow requests fsspec
!pip install pyarrow==14.0.1 requests==2.31.0 fsspec==2024.6.1
!pip install cudf-cu12 gcsfs google-colab ibis-framework

In [ ]:
!pip install transformers datasets torch seqeval pandas

In [ ]:
import json
import os
import pandas as pd
import pyarrow.parquet as pq
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import classification_report
from seqeval.metrics import classification_report, f1_score
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AdamW, get_scheduler
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import DataCollatorForTokenClassification

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Carregar Dataset

Carregar tadaset (presente nos arquivos **news_data.parquet** e **annot.parquet**. Estes arquivos são obtidos após a conversão dos dados dos arquivos json para parquet)

In [ ]:
news_dataset = Dataset.from_parquet('news_data.parquet')
annotated_dataset = Dataset.from_parquet('annot.parquet')

print("Head do dataset puro:")
print(news_dataset[:5])
print("Head do dataset anotado:")
print(annotated_dataset[:5])

print("Estrutura do dataset:")
print(news_dataset)
print("Estrutura do dataset anotado:")
print(annotated_dataset)

## Tratamento e preparação dos dados para o treinamento

Primeiro, garantimos a integridade dos rótulos (ner_tags) para que estejam configurados em string. Garantimos anteriormente a integridade para o uso das ferramentas da bilblioteca dataset da HuggingFace, pois ela trabalha com arquivos no formato da Apache Arrow, e por isso estamos utilizando dados em parquet.
Mapeamos os rótulos de string para IDs numéricos e montamos um Tokenizer e colocamos o acesso ao modelo BERT large, disponibilizado na HuggingFace. Usamos o BERTimbau por ele foi treinado para ser utilizado em problemas de Token Classification em português.
Após isso, alinhamos as labels e tokenizamos o dataset do BERT.

In [ ]:
def ensure_str_labels(dataset):
    def to_str_labels(batch):
        batch['ner_tags'] = [[str(label) for label in labels] for labels in batch['ner_tags']]
        return batch
    return dataset.map(to_str_labels, batched=True)

annotated_dataset = ensure_str_labels(annotated_dataset)

label2id = {label: i for i, label in enumerate(set(tag for tags in annotated_dataset['ner_tags'] for tag in tags))}
id2label = {i: label for label, i in label2id.items()}

bert_model_name = 'neuralmind/bert-large-portuguese-cased'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForTokenClassification.from_pretrained(bert_model_name, num_labels=len(label2id))

def tokenize_and_align_labels(examples):
    tokenized_inputs = bert_tokenizer(
        examples['tokens'],
        truncation=True,
        padding='max_length',
        max_length=512,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  #Ignorar tokens especiais
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])  #Converter rótulo de string para ID
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

tokenized_datasets_bert = annotated_dataset.map(tokenize_and_align_labels, batched=True)

print("Primeiras 5 linhas do dataset tokenizado:")
print(tokenized_datasets_bert[:5])


## Treinamento e Avaliação do modelo

Aqui realizamos o treinamento do nosso modelo e efetuamos a avaliação, para verificar se tudo correu da forma esperada. Primeiro, preparamos os dados para o DataLoader. Configuramos o tamanho do batch (coloquei em 4 porque antes ocorreram problemas com a GPU). Após isso, montamos o DataLoader para treinamento e avaliação, efetuamos as configurações, como o número de épocas (deixei em 10 para ele conseguir ter contato com os outros rótulos que não fossem NULL (O) ), e realizamos o treinamento. Após o treinamento, avaliamos a precisão do modelo, usamos f1-score para ver como foi a performance, assim como a sua precisão, e salvamos o modelo e o tokenizer em uma pasta separada, pois os usaremos para rotular o restante do dataset.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(x['input_ids']) for x in batch])
    attention_mask = torch.stack([torch.tensor(x['attention_mask']) for x in batch])
    labels = torch.stack([torch.tensor(x['labels']) for x in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

batch_size = 4

train_dataloader = DataLoader(tokenized_datasets_bert, shuffle=True, batch_size=batch_size, collate_fn=collate_fn)
eval_dataloader = DataLoader(tokenized_datasets_bert, batch_size=batch_size, collate_fn=collate_fn)

optimizer = AdamW(bert_model.parameters(), lr=2e-5)
num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

def train(model, dataloader, optimizer, lr_scheduler):
    model.train()
    for batch in tqdm(dataloader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        del batch, outputs, loss
        torch.cuda.empty_cache()

def evaluate(model, dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    for batch in tqdm(dataloader, desc="Evaluation"):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch['labels']

        all_predictions.extend(predictions.cpu().numpy().tolist())
        all_labels.extend(labels.cpu().numpy().tolist())

        del batch, outputs, predictions, labels
        torch.cuda.empty_cache()

    return all_predictions, all_labels

bert_model.to(device)

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(bert_model, train_dataloader, optimizer, lr_scheduler)

predictions, labels = evaluate(bert_model, eval_dataloader)

flat_true_labels = [item for sublist in true_labels for item in sublist]
flat_true_predictions = [item for sublist in true_predictions for item in sublist]

true_predictions = [
    [id2label[pred] for pred, label in zip(prediction, label) if label != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [id2label[label] for pred, label in zip(prediction, label) if label != -100]
    for prediction, label in zip(predictions, labels)
]

flat_true_labels = [item for sublist in true_labels for item in sublist]
flat_true_predictions = [item for sublist in true_predictions for item in sublist]

results = classification_report(flat_true_labels, flat_true_predictions, output_dict=True)
print("Evaluation Results:", results)

model_save_path = './saved_model/model'
bert_model.save_pretrained(model_save_path)

print(f"Modelo salvo")


In [ ]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
tokenizer_save_path = './saved_model/tokenizer'
tokenizer.save_pretrained(tokenizer_save_path)

## Rotulação dos dados restantes

Agora que temos um modelo já salvo, utilizamos o modelo e o Tokenizer para rotular o restante dos dados do dataset, que não estavam rotulados. Como o modelo foi refinado anteriormente, agora utilizamos o modelo para rotular os dados e salvar em um arquivo parquet, para tentar preservar sua integridade.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_save_path = './saved_model/model'
tokenizer_save_path = './saved_model/tokenizer'
tokenizer = BertTokenizer.from_pretrained(tokenizer_save_path)
model = BertForTokenClassification.from_pretrained(model_save_path).to(device)

id2label = model.config.id2label

def label_text(text):
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**encoding)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).squeeze().cpu().tolist()
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'].squeeze().cpu().tolist())
    labels = [id2label[pred] for pred in predictions]
    return list(zip(tokens, labels))

news_data = pd.read_parquet('news_data.parquet')

results = []
for index, row in news_data.iterrows():
    labeled_text = label_text(row['content'])
    results.append({'title': row['title'], 'url': row['url'], 'date': row['date'], 'content': labeled_text})

df_results = pd.DataFrame(results)

df_results.to_parquet('labeled_news_data.parquet', index=False)

print("Dataset rotulado e salvo com sucesso!")

## Conversão para outras formas

Por fim, converti os arquivos para ler em outras formas os dados e poder prosseguir com as próximas etapas. O Label-Studio não lê arquivos em parquet, por exemplo, então eu precisava deixar em outros formatos para poder fazer o Pos Annotation por lá.

In [ ]:
parquet_file = 'labeled_news_data.parquet'
df = pd.read_parquet(parquet_file)

df.to_json('labeled_news_data.json', orient='records', lines=True, force_ascii=False, indent=4)
print(f"JSON salvo)

df.to_csv('labeled_news_data.csv', index=False)
print(f"CSV salvo")

df.to_csv('labeled_news_data.txt', index=False, sep='\t')
print("TXT salvo")
